# Computational analysis

For this analysis, three datasets are required. These are;

 1. Kenya boundary shapefile
 2. Kenya Population count by spatial distribution

We will need some packages to get started.

In [ ]:
import os                 # for basic operating system functions
import rasterio
import warnings
import numpy as np
import geopandas as gpd   # For working with geospatial dataframes
import matplotlib.pyplot as plt
import pandas as pd       # To load .csv data as a dataframe
import seaborn as sns
from rasterio.mask import mask
from rasterstats import zonal_stats
from shapely.geometry import MultiPolygon
warnings.filterwarnings('ignore')
from colorspacious import cspace_converter

## 1. Process Kenya boundary 

First we obtain a csv country metadafile containing information about 250 territories and store it in a data directory

In [ ]:
os.getcwd()

In [ ]:
os.chdir("C:/Users/bmwan/Desktop/5.2/Link Budget")

In [ ]:
metadata_file = os.path.join('data', 'countries.csv')

Next we create a function that read the country of interest, process the boundary information and store it in a folder named by the country's **"ISO3"** code as well as create regions subfolder to contain the processed shapefiles. For instance, Kenya will be **"KEN"**


In [ ]:
def country_directory(csv_filename, country_iso3):
    """
    Create country folder and regions subfolder within it.
    
    Parameters
    ----------
    csv_filename : string
        Name of the country metadata file.
    country_iso3 : string
        Country iso3 to be processed.
        
    Returns
    -------
    output : list
        List containing country ISO3 and gid_region
    """
    countries = pd.read_csv(csv_filename, encoding = 'latin-1')
    
    output = []
    
    for idx, country in countries.iterrows():
        
        if not country['iso3'] == country_iso3: #If the current country iso3 does not match the entered iso3...
            continue                            #continue in the loop to the next country

        else:   
            iso3 = country['iso3']
            gid_region = country['gid_region']
            country_name = country['country']
        
        output.append(iso3)
        output.append(gid_region)
        
        country_folder_path = os.path.join('data', 'processed', iso3) #Create folder called "processed" to store
        if not os.path.exists(country_folder_path):                   #country folder
            os.makedirs(country_folder_path)                          #Only create folder if it doest exist already
            
        regions_folder_path = os.path.join('data', 'processed', iso3, 'regions') #Create regions folder within
        if not os.path.exists(regions_folder_path):
            os.makedirs(regions_folder_path)

        print('processing {}'.format(country_name))      
    
    return output

We then use the function to create the folder for Kenya

In [ ]:
country_directory(metadata_file, 'KEN')

Next we download global administrative maps data from https://gadm.org/download_world.html for the whole world. The current GADM version is 4.1 consisting of 400,276 administrative areas globally.

The data is downloaded as a single database in GeoPackage format and stored in the folder called **"raw"**

We then specify the location of the data

In [ ]:
boundary_data = os.path.join('data', 'raw', 'gadm36_1.shp') 

We create a function that read the global administrative boundaries, select a country that we specify using the **"ISO3"** code process the boundaries and store in the corresponding folder.  

In [ ]:
def country_boundary(boundary_data_path, country_iso3, gid_level):
    """
    Process country shapefile.
    
    Parameters
    ----------
    boundary_data_path : string
        Location of the GADM boundary data.
    country_iso3 : string
        Country iso3 to be processed.
    gid_level: string
        GID boundary spatial level to process
        
    Returns
    -------
    none  
    """
    global_boundaries = gpd.read_file(boundary_data_path)
    
    iso3 = country_directory(metadata_file, country_iso3)[0]
    gid_region = country_directory(metadata_file, country_iso3)[1]
    
    country_boundaries = global_boundaries[global_boundaries[gid_level] == iso3]
    gid_level_name = country_boundaries['NAME_1']
    
    filename = 'gadm_{}.shp'.format(gid_region)
    path_out = os.path.join('data', 'processed', iso3, 'regions', filename)
    country_boundaries.to_file(path_out)
    
    print('processing {}'.format(gid_level_name))
    
    return None

We then apply our function for Kenya, (*'ISO3'* = *'KEN'*)

In [ ]:
country_boundary(boundary_data, 'KEN', 'GID_0')

## 2. Kenya Population count by spatial distribution

Next, we need to download and process population data

The population count data is downloaded from WorldPop website (https://hub.worldpop.org/geodata/listing?id=64). 

The data downloaded is the unconstrained global mosaics 2000-2020 for 2020. The layer represent the estimated total number of people per grid-cell, in Geotiff format at a resolution of 30 arc (approximately 1km at the equator), in the WGS84 Geographic Coordinate System.

The downloaded data is stored in the *"raw"* folder.

We specify the location of the data and load the layer to raster input-output (rasterio) object

#### Preliminary data check 

In [ ]:
filename = ("C:/Users/bmwan/Desktop/5.2/Link Budget/data/raw/ppp_2020_1km_Aggregated.tif")
folder = os.path.join('data', 'raw', 'worldpop')
path_population = os.path.join(folder, filename)

These set of cells are just to ensure that both the data layers are working properly as well as checking out their contents.


In [ ]:
my_raster_image = rasterio.open(path_population)
my_raster_image

We can then access the Python API interface for the .tif population layer file based on the Numpy N-dimensional arrays

<div class="alert alert-block alert-warning">

<b>!! Attention !!</b> This my take long since it is loading the whole world!.

</div>

In [ ]:
data = my_raster_image.read(1)
print(data)

We now query our population raster layer using the Kenyan boundary shapefile processed in section 1.

Let us load the processed Kenyan shapefile

In [ ]:
filename = 'gadm_1.shp'
path_boundary = os.path.join('data', 'processed', 'KEN', 'regions', filename)
boundaries = gpd.read_file(path_boundary, crs = 'epsg:4326')
boundaries

#### Process Kenyan population layer

We now write a function that load the data layer, affine function, eliminate negative cell numbers, sum all the cells within the boundary of interest and return a csv file for plotting.

In [ ]:
def process_population_tif(data_name, grid_level):
    """
    Process population layer.
    
    Parameters
    ----------
    data_name: string
        Filename of the population raster layer
    gid_level: string
        GID boundary spatial level to process
        
    Returns
    -------
    output: dictionary.
        Dictionary containing the country population and grid level
    """
    output = []
    for idx, boundary in boundaries.iterrows():
        
        print('Working on {}'.format(boundary['NAME_1']))
        
        with rasterio.open(data_name) as src:
            
            affine = src.transform
            array = src.read(1)
            array[array <= 0] = 0
            
            population = [i['sum'] for i in zonal_stats(
                boundary['geometry'], array, nodata = 255,
                stats = ['sum'], affine = affine)][0]
            output.append({
                'NAME_1': boundary['NAME_1'],
                'GID_1': boundary[grid_level],
                'population': population
            })
            
    return output

We now apply the function to our population layer



In [ ]:
filename = ("C:/Users/bmwan/Desktop/5.2/Link Budget/data/raw/ppp_2020_1km_Aggregated.tif")
folder = os.path.join('data', 'raw', 'worldpop')
path_population = os.path.join(folder, filename)
pop = process_population_tif(path_population, 'GID_1')

And then we store our results as csv file in the *processed/KEN* folder

In [ ]:
pop_results = pd.DataFrame(pop)
path_out = os.path.join('data', 'processed', 'KEN', 'population.csv')
pop_results.to_csv(path_out)

In [ ]:
path_out = os.path.join("C:/Users/bmwan/Desktop/5.2/Link Budget/data/processed/KEN/population.csv")
pop_results = pd.read_csv(path_out)

#### Plotting population map 

In [ ]:
pop_results.head()

First, we import country boundary.

In [ ]:
#import our boundaries data
filename = 'regions_1_KEN.shp'
path_in = os.path.join("C:/Users/bmwan/Desktop/5.2/Link Budget/data/processed/KEN/regions/", filename) 
country_boundaries = gpd.read_file(path_in)
country_boundaries.head(5)

In [ ]:
boundaries.head(5)

In [ ]:
boundaries.columns

In [ ]:
boundaries['GID_1','NAME_1_x','geometry']
boundaries['area'] = boundaries.geometry.area
boundaries.to_csv("C:/Users/bmwan/Desktop/5.2/Link Budget/data/processed/Boundaries/b_1.csv")

In [ ]:
country_boundaries = country_boundaries[['NAME_1', 'GID_1', 'geometry']]
country_boundaries.head(5)

In [ ]:
country_boundaries.head()

And then we import the previously-saved population csv file.

In [ ]:
boundaries = country_boundaries.merge(pop_results, left_on = 'GID_1', right_on = 'GID_1')
boundaries = boundaries.drop(['NAME_1_y'], axis = 1)
boundaries.to_csv("population.csv")
boundaries.head(5)

In [ ]:
#import our population data
filename = "C:/Users/bmwan/Desktop/5.2/Link Budget/data/processed/KEN/population.csv"
path_in = os.path.join('data', 'processed', 'KEN', filename) 
data = pd.read_csv(path_in)

First, we'll show the range of each colormap

In [ ]:
cmaps = {}

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(category, cmap_list):
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.22
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(6.4, figh))
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.15 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=14)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=plt.colormaps[name])
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)
    for ax in axs:
        ax.set_axis_off()
    cmaps[category] = cmap_list

We then write a function to plot the data.

<div class="alert alert-block alert-info">
    
<b>Note:</b> You need to manually adjust the bins and labels inside the function depending on the distribution of your dataset.

</div>

In [ ]:
def plot_population(country_boundaries, csv_data, gid_level, iso3):
    """
    Plots the country's population at sub-region level.
    
    Parameters
    ----------
    country_boundaries : string
        Name of the country boundary .shp file
    csv_data : string
        Name of the csv file containing the population data
        at sub-regional level
    gid_level : string
        Name of the GID level
    country_iso3 = string
        ISO3 of the country
    
    Return
    ------
    fig : python object
        A map of population distribution. 
    """
    plot_color_gradients ('Qualitative',
                     ['Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2',
                      'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b',
                      'tab20c'])
    boundaries = country_boundaries.merge(csv_data, left_on = gid_level, #merge our population data onto our boundaries
                                  right_on = gid_level)
    
    bins = [-1e6, 10000, 20000, 30000, 40000, 50000, 60000,        #define value bins and then labels for each one
            70000, 80000, 90000, 100000, 110000, 120000, 130000,      # 24
            140000, 150000, 160000, 170000, 180000, 190000, 200000,
            210000, 220000, 230000]
    
    labels = [
                '1-10k','10-20k','20-30k', '30-40k',      #46
                '40-50k', '50-60k','60-70k', '70-80k', 
                '80-90k', '90-100k',
                '100-110k', '110-120k','120-130k', '130-140k',
                '140-150k', '150-160k', '160-170k', '170-180k',
                '180-190k', '190-200k','200-210k', '210-220k', 
                '>220k'
                ]
     
    boundaries['bin'] = pd.cut(                                   #create a new variable with our bin labels
        boundaries['population'], 
        bins = bins,
        labels = labels
    )   
    
    sns.set(font_scale = 0.9)                                     #open a new seaborn figure
    fig, ax = plt.subplots(1, 1, figsize = (8, 8))

    base = boundaries.plot(column = 'bin', ax = ax,               #now plot our data using pandas plot
                           cmap = 'Blues', linewidth = 0, 
                           legend = True, antialiased = False)
    #allocate a plot title 
    n = len(boundaries)
    name = ' Kenyan Population by Sub-Region(n={})'.format(n)
    fig.suptitle(name)
    
    filename = 'kenyan_population_2.png'
    folder_out = os.path.join('data', 'processed', iso3)    
    if not os.path.exists(folder_out):  
        os.makedirs(folder_out) 
    path_out = os.path.join(folder_out, filename)
    fig.savefig(path_out)   
    plt.close(fig)

    fig
    
    return fig

In [ ]:
plot_population(country_boundaries, pop_results, 'GID_1', 'KEN')